In [1]:

import pandas as pd
import warnings
import requests
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


warnings.simplefilter(action='ignore', category=FutureWarning)

from sqlalchemy import create_engine
import numpy as np
import pickle
from scipy import stats
from tqdm import tqdm
import datetime as dt
from retrying import retry

from utils.env import *
from utils import avgNav, commonMetric, query, thematicManager as tm, tableManager

from oauth2client.service_account import ServiceAccountCredentials

In [2]:
@retry(stop_max_attempt_number=3)
def execute_query(sql_query, engine):
    return pd.read_sql_query(sql_query, con=engine)


def get_adr_stock_ids(ver, cnx):
    adr_stock_ids = []
    stockinfo_qr = f'''SELECT * FROM `{ver}_stock_infos`'''
    stockinfo = pd.read_sql_query(stockinfo_qr, con=cnx)
    adr_stock_ids.extend(stockinfo[stockinfo.adr == 1].jittaStockId.to_list())
    
    return adr_stock_ids



In [3]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time

def create_session_with_retries(retries, backoff_factor, status_forcelist):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def sandbox_v2(fid, jid, bid, uid, scope):
    url = f"http://192.168.152.53:3000/api/v2/formulas/{fid}/results"
    params = {
        "jitta_stock_ids": jid,
        "buildVersionId": bid,
        "sector": "ALL",
        "industry": "ALL",
        "scopeType": scope,
        "limit": "",
        "skip": ""
    }
    headers = {
        "universe-id": uid
    }

    retries = 5
    backoff_factor = 0.3
    status_forcelist = [500, 502, 503, 504]

    session = create_session_with_retries(retries, backoff_factor, status_forcelist)

    try:
        response = session.get(url, params=params, headers=headers, timeout=120)
        if response.status_code == 200:
            time.sleep(0.5)
            data = response.json()
            return data
        else:
            print("Failed to retrieve data. Status code:", response.status_code)
            return None
    except requests.exceptions.Timeout:
        print("The request timed out")
        return None
    except requests.exceptions.RequestException as e:
        # Handle other possible exceptions
        print(f"An error occurred: {e}")
        return None


# def sandbox_v2(fid, jid, bid, uid, scope):
#     url = f"http://192.168.152.53:3000/api/v2/formulas/{fid}/results"
#     params = {
#         "jitta_stock_ids": jid,
#         "buildVersionId": bid,
#         "sector": "ALL",
#         "industry": "ALL",
#         "scopeType": scope,
#         "limit": "",
#         "skip": ""
#     }
#     headers = {
#         "universe-id": uid
#     }

#     response = requests.get(url, params=params, headers=headers, timeout=120)

#     if response.status_code == 200:
#         time.sleep(1)
#         data = response.json()
#         return data
#     else:
#         print("Failed to retrieve data. Status code:", response.status_code)
#         return None
    

def rolling_quantile(df, column, quantile, window_size):
    expanding_window_quantile = df[column].expanding(min_periods=1).quantile(quantile)
    rolling_window_quantile = df[column].rolling(window=window_size, min_periods=window_size).quantile(quantile)
    return expanding_window_quantile.combine_first(rolling_window_quantile)


def get_clean_jitta_stock_ids(jitta_score_date_like, list_adr, list_etf, skip_jid, cnx, ver):
    clean_jitta_stock_ids_dict = {}
    
    for i, date in tqdm(enumerate(jitta_score_date_like)):
        if len(list_adr) == 1:
            top_jitta_score_qr = f'''
                SELECT * FROM `{ver}_jitta_score_price$monthly$1_Bh-KI69fC`
                WHERE seen LIKE '{date}'
                AND `jittaStockId` NOT IN {tuple(list_etf)}
                AND `jittaStockId` NOT IN {tuple(skip_jid)}
                AND `jittaStockId` !=  {list_adr[0]}
                ORDER BY `value` DESC
                LIMIT 200;
                '''
        elif len(list_adr) == 0:
            top_jitta_score_qr = f'''
                SELECT * FROM `{ver}_jitta_score_price$monthly$1_Bh-KI69fC`
                WHERE seen LIKE '{date}'
                AND `jittaStockId` NOT IN {tuple(list_etf)}
                AND `jittaStockId` NOT IN {tuple(skip_jid)}
                ORDER BY `value` DESC
                LIMIT 200;
                '''
        else:
            top_jitta_score_qr = f'''
                SELECT * FROM `{ver}_jitta_score_price$monthly$1_Bh-KI69fC`
                WHERE seen LIKE '{date}'
                AND `jittaStockId` NOT IN {tuple(list_etf)}
                AND `jittaStockId` NOT IN {tuple(skip_jid)}
                AND `jittaStockId` NOT IN {tuple(list_adr)}
                ORDER BY `value` DESC
                LIMIT 200;
                '''

        df = pd.read_sql_query(top_jitta_score_qr, con=cnx)
        df.seen = pd.to_datetime(df.seen)
        latest_seen_df = df.loc[df.groupby('jittaStockId')['seen'].idxmax()]
        latest_seen_df_sorted = latest_seen_df.sort_values(by='value', ascending=False)[:50]

        # result = result[result.value > 7] # filter score > 7
        # display(latest_seen_df_sorted)
        
        jitta_stock_ids = latest_seen_df_sorted.jittaStockId.to_list()
        clean_jitta_stock_ids = [id for id in jitta_stock_ids]
        
        # Appending to the dictionary
        clean_jitta_stock_ids_dict[date] = clean_jitta_stock_ids
    
    return clean_jitta_stock_ids_dict

In [4]:

def computer_each_dataversion(clean_jitta_stock_ids_dict, ver, fid, bid, scope):
    cache = {}
    pe_zone = {}
    
    for date in clean_jitta_stock_ids_dict.keys():
        top50_each_date = clean_jitta_stock_ids_dict[str(date)]
        list_pe_catagory2 = []
        
        if len(top50_each_date) == 0:
            continue
        
        for stock_id in tqdm(top50_each_date):
            if stock_id in cache:
                df1 = cache[stock_id]
                SKIP = True
            else:
                SKIP = False
                
            if not SKIP:
                resp = sandbox_v2(fid=fid, jid=stock_id, bid=bid, uid=ver, scope=scope)
                value = resp['data'][0]['value']
                
                pe_df = pd.DataFrame(value)
                pe_df.set_index('seen', inplace=True)
                pe_df = pe_df[~pe_df.index.duplicated(keep='last')]  
                pe_df.index = pd.to_datetime(pe_df.index)
                pe_df = pe_df.resample('D').last()
                
                # PE Calculation
                df1 = pe_df[['v']].copy()
                df1.drop(df1[df1['v'] == "N/A"].index, inplace=True)
                df1.dropna(inplace=True)
                df1 = df1.astype(float)
                df1['jitta_stock_id'] = stock_id
                                
                df1['rolling_10th'] = rolling_quantile(df1, 'v', 0.1, 10*255)  # rolling daily
                df1['rolling_20th'] = rolling_quantile(df1, 'v', 0.2, 10*255)
                df1['rolling_30th'] = rolling_quantile(df1, 'v', 0.3, 10*255)
                df1['rolling_40th'] = rolling_quantile(df1, 'v', 0.4, 10*255)
                df1['rolling_50th'] = rolling_quantile(df1, 'v', 0.5, 10*255)
                df1['rolling_60th'] = rolling_quantile(df1, 'v', 0.6, 10*255)
                df1['rolling_70th'] = rolling_quantile(df1, 'v', 0.7, 10*255)
                df1['rolling_80th'] = rolling_quantile(df1, 'v', 0.8, 10*255)
                df1['rolling_90th'] = rolling_quantile(df1, 'v', 0.9, 10*255)

                df1['PE_category2'] = np.where(df1['v'] < df1['rolling_10th'], '0_10',
                                                    np.where((df1['v'] >= df1['rolling_10th']) & (df1['v'] < df1['rolling_20th']), '10_20',
                                                    np.where((df1['v'] >= df1['rolling_20th']) & (df1['v'] < df1['rolling_30th']), '20_30',
                                                    np.where((df1['v'] >= df1['rolling_30th']) & (df1['v'] < df1['rolling_40th']), '30_40',
                                                    np.where((df1['v'] >= df1['rolling_40th']) & (df1['v'] < df1['rolling_50th']), '40_50',
                                                    np.where((df1['v'] >= df1['rolling_50th']) & (df1['v'] < df1['rolling_60th']), '50_60',
                                                    np.where((df1['v'] >= df1['rolling_60th']) & (df1['v'] < df1['rolling_70th']), '60_70',
                                                    np.where((df1['v'] >= df1['rolling_70th']) & (df1['v'] < df1['rolling_80th']), '70_80',
                                                    np.where((df1['v'] >= df1['rolling_80th']) & (df1['v'] < df1['rolling_90th']), '80_90',
                                                    np.where((df1['v'] >= df1['rolling_90th']), '90+', 'Unknown'))))))))))

                df1.index = pd.to_datetime(df1.index) 
                cache[stock_id] = df1
                            
            try:
                pe_category_value2 = df1.loc[(df1.index.year == int(date[:4])) & 
                                             (df1.index.month == int(date[5:7])) &
                                             (df1.index.day == int(date[-2:])), 'PE_category2'].values[-1]
                
                list_pe_catagory2.append(pe_category_value2)
            except Exception as e:
                # print(stock_id, e)   
                list_pe_catagory2.append(np.nan)  
        
        # Calculate how many NaN values are needed
        num_nans_to_append = 50 - len(list_pe_catagory2)
        # Append NaN values if needed
        if num_nans_to_append > 0:
            list_pe_catagory2.extend([np.nan] * num_nans_to_append)
        pe_zone[date] = list_pe_catagory2      
            
    return pe_zone


In [5]:
def display_pe_zone(pe_zone):
    # Create DataFrame
    df = pd.DataFrame(pe_zone)

    # Count occurrences of each value
    value_counts_df = df.apply(pd.Series.value_counts)

    # Reindex to ensure all categories are present
    index_categories = ['0_10', '10_20', '20_30', '30_40', '40_50', '50_60', '60_70', '70_80', '80_90', '90+']
    value_counts_df = value_counts_df.reindex(index_categories)

    # Display the transposed DataFrame for better readability
    display(value_counts_df.T)
    

def write_pe_zone(pe_zone, dataversion):
    # Create DataFrame
    df = pd.DataFrame(pe_zone)

    # Count occurrences of each value
    value_counts_df = df.apply(pd.Series.value_counts)

    # Reindex to ensure all categories are present
    index_categories = ['0_10', '10_20', '20_30', '30_40', '40_50', '50_60', '60_70', '70_80', '80_90', '90+']
    value_counts_df = value_counts_df.reindex(index_categories)

    # Display the transposed DataFrame for better readability
    df1 = value_counts_df.T
    df1.to_csv(f"PE2013_{dataversion}.csv")


In [6]:
list_dataversion = [
    # 'US_2024-05-31',
    # 'HK_2024-05-31',
    # 'CN_2024-05-31',
    # # 'TH_2024-05-01',
    # 'JP_2024-05-31',
    # # 'VN_2024-05-01',

    'UK_2024-08-05',
    
    # 'IN_2024-01-03',
    # 'KR_2024-01-03',
    # 'TW_2024-01-03',
    # 'SG_2024-06-11',
    # 'DE_2024-06-12',/
    # 'AU_2024-06-11',
    # 'CA_2024-01-03',
    ]

jitta_score_date_like = [ 
 '2003-01-%%',
 '2003-02-%%',
 '2003-03-%%',
 '2003-04-%%',
 '2003-05-%%',
 '2003-06-%%',
 '2003-07-%%',
 '2003-08-%%',
 '2003-09-%%',
 '2003-10-%%',
 '2003-11-%%',
 '2003-12-%%',
 '2004-01-%%',
 '2004-02-%%',
 '2004-03-%%',
 '2004-04-%%',
 '2004-05-%%',
 '2004-06-%%',
 '2004-07-%%',
 '2004-08-%%',
 '2004-09-%%',
 '2004-10-%%',
 '2004-11-%%',
 '2004-12-%%',
 '2005-01-%%',
 '2005-02-%%',
 '2005-03-%%',
 '2005-04-%%',
 '2005-05-%%',
 '2005-06-%%',
 '2005-07-%%',
 '2005-08-%%',
 '2005-09-%%',
 '2005-10-%%',
 '2005-11-%%',
 '2005-12-%%',
 '2006-01-%%',
 '2006-02-%%',
 '2006-03-%%',
 '2006-04-%%',
 '2006-05-%%',
 '2006-06-%%',
 '2006-07-%%',
 '2006-08-%%',
 '2006-09-%%',
 '2006-10-%%',
 '2006-11-%%',
 '2006-12-%%',
 '2007-01-%%',
 '2007-02-%%',
 '2007-03-%%',
 '2007-04-%%',
 '2007-05-%%',
 '2007-06-%%',
 '2007-07-%%',
 '2007-08-%%',
 '2007-09-%%',
 '2007-10-%%',
 '2007-11-%%',
 '2007-12-%%',
 '2008-01-%%',
 '2008-02-%%',
 '2008-03-%%',
 '2008-04-%%',
 '2008-05-%%',
 '2008-06-%%',
 '2008-07-%%',
 '2008-08-%%',
 '2008-09-%%',
 '2008-10-%%',
 '2008-11-%%',
 '2008-12-%%',
 '2009-01-%%',
 '2009-02-%%',
 '2009-03-%%',
 '2009-04-%%',
 '2009-05-%%',
 '2009-06-%%',
 '2009-07-%%',
 '2009-08-%%',
 '2009-09-%%',
 '2009-10-%%',
 '2009-11-%%',
 '2009-12-%%',
 '2010-01-%%',
 '2010-02-%%',
 '2010-03-%%',
 '2010-04-%%',
 '2010-05-%%',
 '2010-06-%%',
 '2010-07-%%',
 '2010-08-%%',
 '2010-09-%%',
 '2010-10-%%',
 '2010-11-%%',
 '2010-12-%%',
 '2011-01-%%',
 '2011-02-%%',
 '2011-03-%%',
 '2011-04-%%',
 '2011-05-%%',
 '2011-06-%%',
 '2011-07-%%',
 '2011-08-%%',
 '2011-09-%%',
 '2011-10-%%',
 '2011-11-%%',
 '2011-12-%%',
 '2012-01-%%',
 '2012-02-%%',
 '2012-03-%%',
 '2012-04-%%',
 '2012-05-%%',
 '2012-06-%%',
 '2012-07-%%',
 '2012-08-%%',
 '2012-09-%%',
 '2012-10-%%',
 '2012-11-%%',
 '2012-12-%%',
                         
 '2013-01-%%',
 '2013-02-%%',
 '2013-03-%%',
 '2013-04-%%',
 '2013-05-%%',
 '2013-06-%%',
 '2013-07-%%',
 '2013-08-%%',
 '2013-09-%%',
 '2013-10-%%',
 '2013-11-%%',
 '2013-12-%%',
 '2014-01-%%',
 '2014-02-%%',
 '2014-03-%%',
 '2014-04-%%',
 '2014-05-%%',
 '2014-06-%%',
 '2014-07-%%',
 '2014-08-%%',
 '2014-09-%%',
 '2014-10-%%',
 '2014-11-%%',
 '2014-12-%%',
 '2015-01-%%',
 '2015-02-%%',
 '2015-03-%%',
 '2015-04-%%',
 '2015-05-%%',
 '2015-06-%%',
 '2015-07-%%',
 '2015-08-%%',
 '2015-09-%%',
 '2015-10-%%',
 '2015-11-%%',
 '2015-12-%%',
 '2016-01-%%',
 '2016-02-%%',
 '2016-03-%%',
 '2016-04-%%',
 '2016-05-%%',
 '2016-06-%%',
 '2016-07-%%',
 '2016-08-%%',
 '2016-09-%%',
 '2016-10-%%',
 '2016-11-%%',
 '2016-12-%%',
 '2017-01-%%',
 '2017-02-%%',
 '2017-03-%%',
 '2017-04-%%',
 '2017-05-%%',
 '2017-06-%%',
 '2017-07-%%',
 '2017-08-%%',
 '2017-09-%%',
 '2017-10-%%',
 '2017-11-%%',
 '2017-12-%%',
 '2018-01-%%',
 '2018-02-%%',
 '2018-03-%%',
 '2018-04-%%',
 '2018-05-%%',
 '2018-06-%%',
 '2018-07-%%',
 '2018-08-%%',
 '2018-09-%%',
 '2018-10-%%',
 '2018-11-%%',
 '2018-12-%%',
 '2019-01-%%',
 '2019-02-%%',
 '2019-03-%%',
 '2019-04-%%',
 '2019-05-%%',
 '2019-06-%%',
 '2019-07-%%',
 '2019-08-%%',
 '2019-09-%%',
 '2019-10-%%',
 '2019-11-%%',
 '2019-12-%%',
 '2020-01-%%',
 '2020-02-%%',
 '2020-03-%%',
 '2020-04-%%',
 '2020-05-%%',
 '2020-06-%%',
 '2020-07-%%',
 '2020-08-%%',
 '2020-09-%%',
 '2020-10-%%',
 '2020-11-%%',
 '2020-12-%%',
 '2021-01-%%',
 '2021-02-%%',
 '2021-03-%%',
 '2021-04-%%',
 '2021-05-%%',
 '2021-06-%%',
 '2021-07-%%',
 '2021-08-%%',
 '2021-09-%%',
 '2021-10-%%',
 '2021-11-%%',
 '2021-12-%%',
 '2022-01-%%',
 '2022-02-%%',
 '2022-03-%%',
 '2022-04-%%',
 '2022-05-%%',
 '2022-06-%%',
 '2022-07-%%',
 '2022-08-%%',
 '2022-09-%%',
 '2022-10-%%',
 '2022-11-%%',
 '2022-12-%%',
 '2023-01-%%',
 '2023-02-%%',
 '2023-03-%%',
 '2023-04-%%',
 '2023-05-%%',
 '2023-06-%%',
 '2023-07-%%',
 '2023-08-%%',
 '2023-09-%%',
 '2023-10-%%',
 '2023-11-%%',
 '2023-12-%%'
 ]

# skip_jid
skip_jid = [8173, 2604908, 2259841, 2583555, 2259840, 2259842, 2259843, 2578562, 2573968, 2574007, 2604908]
# list_etf
etf_df = pd.read_json("etf.json", orient='records')
list_etf = pd.DataFrame.from_records(etf_df['US']).jitta_stock_id.to_list()



In [7]:
from datetime import datetime, timedelta
import calendar

# Function to generate dates for a specific month and year
def generate_dates(year, month):
    num_days = calendar.monthrange(year, month)[1]  # Get the number of days in the month
    return [datetime(year, month, day).strftime('%Y-%m-%d') for day in range(1, num_days + 1)]


def main():
    for dataversion in list_dataversion:
        print(dataversion)
        list_adr = get_adr_stock_ids(dataversion, cnx)
        input_dict = get_clean_jitta_stock_ids(jitta_score_date_like, list_adr, list_etf, skip_jid, cnx, ver=dataversion)
        
        # Output dictionary
        output_dict = {}

        for key in input_dict:
            # Extract year and month
            year_month = key.split('-%%')[0]
            year, month = map(int, year_month.split('-'))
            
            # Generate all dates for the month
            dates = generate_dates(year, month)
            
            # Populate the output dictionary with expanded dates
            for date in dates:
                output_dict[date] = input_dict[key]
        
        
        pe_decile = computer_each_dataversion(output_dict, dataversion, fid, bid, scope)
        # display_pe_zone(pe_decile)
        write_pe_zone(pe_decile, dataversion)
        
        
if __name__ == "__main__":
    # fid, bid, scope = '6633b62594330887b8ec65cb', '1NCMPuCYt', 'monthly' # pe6
    fid, bid, scope = '666c7c701b3c7f98461c8525', 'Oed_jIRT9', 'daily'

    main()

UK_2024-08-05


252it [01:33,  2.70it/s]
100%|██████████| 50/50 [00:00<00:00, 880.35it/s]
